#### Feature Engineering  

Dataset: 
- _calls_clean.csv_
- _internet_clean.csv_
- _messages_clean.csv_
- _plans_clean.csv_
- _users_clean.csv_

Author: Luis Sergio Pastrana Lemus  
Date: 2025-05-14

# Feature engineering – Purchasing Activity Dataset

## __1. Libraries__

In [1]:
from IPython.display import display, HTML
from functools import partial
import numpy as np
import os
import pandas as pd
from pathlib import Path
import sys

# Define project root dynamically, gets the current directory from whick the notebook belongs and moves one level upper
project_root = Path.cwd().parent

# Add src to sys.path if it is not already
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import function directly (more controlled than import *)
from src import *

## __2. Path to Data file__

In [2]:
# Build route to data file and upload
data_file_path = project_root / "data" / "processed" / "clean"
df_calls = load_dataset_from_csv(data_file_path, "calls_clean.csv", sep=',', header='infer')
df_internet = load_dataset_from_csv(data_file_path, "internet_clean.csv", sep=',', header='infer')
df_messages = load_dataset_from_csv(data_file_path, "messages_clean.csv", sep=',', header='infer')
df_plans = load_dataset_from_csv(data_file_path, "plans_clean.csv", sep=',', header='infer')
df_users = load_dataset_from_csv(data_file_path, "users_clean.csv", sep=',', header='infer')


In [3]:
# Format notebook output
format_notebook()

## 3 __Functions__

In [114]:
# Function for calculating the revenue per user
def revenue(row, revenue_type='total', columns=None): 
    plans = {
        'surf': {
            'base_fee': 20, 'minutes_limit': 500, 'messages_limit': 50, 'mb_limit': 15360,
            'minute_over': 0.03, 'message_over': 0.03, 'mb_over': 0.01
        },
        'ultimate': {
            'base_fee': 70, 'minutes_limit': 3000, 'messages_limit': 1000, 'mb_limit': 30720,
            'minute_over': 0.01, 'message_over': 0.01, 'mb_over': 0.007
        }
    }

    plan = plans[row['plan']]
    minutes = row[columns[0]]
    traffic = row[columns[1]]
    messages = row[columns[2]]

    income_duration = max(0, minutes - plan['minutes_limit']) * plan['minute_over']
    income_traffic = max(0, traffic - plan['mb_limit']) * plan['mb_over']
    income_messages = max(0, messages - plan['messages_limit']) * plan['message_over']

    if revenue_type == 'duration':
        return plan['base_fee'] + income_duration
    elif revenue_type == 'messages':
        return plan['base_fee'] + income_messages
    elif revenue_type == 'traffic':
        return plan['base_fee'] + income_traffic
    elif revenue_type == 'total':
        return plan['base_fee'] + income_duration + income_traffic + income_messages


## 4 __Casting to data types__

### 3.1 Casting to string data type

In [5]:
# df_users 'first_name' to string
df_users['first_name'] = df_users['first_name'].astype('string')
df_users['first_name'].dtypes

string[python]

In [6]:
# df_users 'last_name' to string
df_users['last_name'] = df_users['last_name'].astype('string')
df_users['last_name'].dtypes

string[python]

In [7]:
# df_users 'city to string
df_users['city'] = df_users['city'].astype('string')
df_users['city'].dtypes

string[python]

### 3.2 Casting to category data type

In [8]:
# df_plans 'plan_name' to category
df_plans['plan_name'] = df_plans['plan_name'].astype('category')
df_plans['plan_name'].dtypes


CategoricalDtype(categories=['surf', 'ultimate'], ordered=False, categories_dtype=object)

In [9]:
# df_users 'plan' to category
df_users['plan'] = df_users['plan'].astype('category')
df_users['plan'].dtypes

CategoricalDtype(categories=['surf', 'ultimate'], ordered=False, categories_dtype=object)

### 3.3 Casting to datetime data type

In [10]:
# df_calls 'call_date' to datetime
df_calls = normalize_datetime(df_calls, include=['call_date'], frmt='%Y-%m-%d')
df_calls.dtypes

call_id                   object
user_id                    int64
call_date    datetime64[ns, UTC]
duration                 float64
dtype: object

In [11]:
# df_internet 'session_date' to datetime
df_internet = normalize_datetime(df_internet, include=['session_date'], frmt='%Y-%m-%d')
df_internet.dtypes

session_id                   object
user_id                       int64
session_date    datetime64[ns, UTC]
mb_used                     float64
dtype: object

In [12]:
# df_messages 'messages_date' to datetime
df_messages = normalize_datetime(df_messages, include=['message_date'], frmt='%Y-%m-%d')
df_messages.dtypes

message_id                   object
user_id                       int64
message_date    datetime64[ns, UTC]
dtype: object

In [13]:
# df_users 'reg_date' to datetime
df_users = normalize_datetime(df_users, include=['reg_date'], frmt='%Y-%m-%d')


In [14]:
# df_users 'churn_date' to datetime
df_users = normalize_datetime(df_users, include=['churn_date'], frmt='%Y-%m-%d')
df_users.dtypes

user_id                     int64
first_name         string[python]
last_name          string[python]
age                         int64
city               string[python]
reg_date      datetime64[ns, UTC]
plan                     category
churn_date    datetime64[ns, UTC]
dtype: object

## 4. Feature Engineering

### 4.1 Datasets

#### 4.1.1 Calls

In [15]:
df_calls

,call_id,user_id,call_date,duration
0,1000_93,1000,2018-12-27 00:00:00+00:00,8.52
1,1000_145,1000,2018-12-27 00:00:00+00:00,13.66
2,1000_247,1000,2018-12-27 00:00:00+00:00,14.48
3,1000_309,1000,2018-12-28 00:00:00+00:00,5.76
4,1000_380,1000,2018-12-30 00:00:00+00:00,4.22
...,...,...,...,...
137730,1499_199,1499,2018-11-21 00:00:00+00:00,8.72
137731,1499_200,1499,2018-10-20 00:00:00+00:00,10.89
137732,1499_201,1499,2018-09-21 00:00:00+00:00,8.12
137733,1499_202,1499,2018-10-10 00:00:00+00:00,0.37


In [16]:
# Each individual call is rounded up: even if the call lasted only one second
df_calls['duration'] = np.ceil(df_calls['duration'])

In [17]:
# Add Month
df_calls['month'] = df_calls['call_date'].dt.month

In [18]:
# Add Day
df_calls['day'] = df_calls['call_date'].dt.day

In [19]:
df_calls

,call_id,user_id,call_date,duration,month,day
0,1000_93,1000,2018-12-27 00:00:00+00:00,9.0,12,27
1,1000_145,1000,2018-12-27 00:00:00+00:00,14.0,12,27
2,1000_247,1000,2018-12-27 00:00:00+00:00,15.0,12,27
3,1000_309,1000,2018-12-28 00:00:00+00:00,6.0,12,28
4,1000_380,1000,2018-12-30 00:00:00+00:00,5.0,12,30
...,...,...,...,...,...,...
137730,1499_199,1499,2018-11-21 00:00:00+00:00,9.0,11,21
137731,1499_200,1499,2018-10-20 00:00:00+00:00,11.0,10,20
137732,1499_201,1499,2018-09-21 00:00:00+00:00,9.0,9,21
137733,1499_202,1499,2018-10-10 00:00:00+00:00,1.0,10,10


In [20]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "calls_feature.csv"

df_calls.to_csv(processed_path, index=False)

#### 4.1.2 Internet

In [21]:
df_internet

,session_id,user_id,session_date,mb_used
0,1000_13,1000,2018-12-29 00:00:00+00:00,89.86
1,1000_204,1000,2018-12-31 00:00:00+00:00,0.00
2,1000_379,1000,2018-12-28 00:00:00+00:00,660.40
3,1000_413,1000,2018-12-26 00:00:00+00:00,270.99
4,1000_442,1000,2018-12-27 00:00:00+00:00,880.22
...,...,...,...,...
104820,1499_215,1499,2018-10-20 00:00:00+00:00,218.06
104821,1499_216,1499,2018-12-30 00:00:00+00:00,304.72
104822,1499_217,1499,2018-09-22 00:00:00+00:00,292.75
104823,1499_218,1499,2018-12-07 00:00:00+00:00,0.00


In [22]:
# Add Month
df_internet['month'] = df_internet['session_date'].dt.month

In [23]:
# Add Day
df_internet['day'] = df_internet['session_date'].dt.day

In [24]:
df_internet

,session_id,user_id,session_date,mb_used,month,day
0,1000_13,1000,2018-12-29 00:00:00+00:00,89.86,12,29
1,1000_204,1000,2018-12-31 00:00:00+00:00,0.00,12,31
2,1000_379,1000,2018-12-28 00:00:00+00:00,660.40,12,28
3,1000_413,1000,2018-12-26 00:00:00+00:00,270.99,12,26
4,1000_442,1000,2018-12-27 00:00:00+00:00,880.22,12,27
...,...,...,...,...,...,...
104820,1499_215,1499,2018-10-20 00:00:00+00:00,218.06,10,20
104821,1499_216,1499,2018-12-30 00:00:00+00:00,304.72,12,30
104822,1499_217,1499,2018-09-22 00:00:00+00:00,292.75,9,22
104823,1499_218,1499,2018-12-07 00:00:00+00:00,0.00,12,7


In [25]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "internet_feature.csv"

df_internet.to_csv(processed_path, index=False)

#### 4.1.3 Messages

In [26]:
df_messages

,message_id,user_id,message_date
0,1000_125,1000,2018-12-27 00:00:00+00:00
1,1000_160,1000,2018-12-31 00:00:00+00:00
2,1000_223,1000,2018-12-31 00:00:00+00:00
3,1000_251,1000,2018-12-27 00:00:00+00:00
4,1000_255,1000,2018-12-26 00:00:00+00:00
...,...,...,...
76046,1497_526,1497,2018-12-24 00:00:00+00:00
76047,1497_536,1497,2018-12-24 00:00:00+00:00
76048,1497_547,1497,2018-12-31 00:00:00+00:00
76049,1497_558,1497,2018-12-24 00:00:00+00:00


In [27]:
# Add Month
df_messages['month'] = df_messages['message_date'].dt.month

In [28]:
# Add Day
df_messages['day'] = df_messages['message_date'].dt.day

In [29]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "messages_feature.csv"

df_messages.to_csv(processed_path, index=False)

#### 4.1.4 Users

In [30]:
df_users

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
0,1000,anamaria,bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",2018-12-24 00:00:00+00:00,ultimate,NaT
1,1001,mickey,wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",2018-08-13 00:00:00+00:00,surf,NaT
2,1002,carlee,hoffman,36,"las_vegas_henderson_paradise,_nv_msa",2018-10-21 00:00:00+00:00,surf,NaT
3,1003,reynaldo,jenkins,52,"tulsa,_ok_msa",2018-01-28 00:00:00+00:00,surf,NaT
4,1004,leonila,thompson,40,"seattle_tacoma_bellevue,_wa_msa",2018-05-23 00:00:00+00:00,surf,NaT
...,...,...,...,...,...,...,...,...
495,1495,fidel,sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",2018-09-04 00:00:00+00:00,surf,NaT
496,1496,ariel,shepherd,49,"new_orleans_metairie,_la_msa",2018-02-20 00:00:00+00:00,surf,NaT
497,1497,donte,barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",2018-12-10 00:00:00+00:00,ultimate,NaT
498,1498,scot,williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",2018-02-04 00:00:00+00:00,surf,NaT


In [31]:
# Full Name
df_users['full_name'] = df_users['first_name'] + "_" + df_users['last_name']

In [32]:
# Add Month 'reg_date'
df_users['reg_month'] = df_users['reg_date'].dt.month

In [33]:
# Add Day 'reg_date'
df_users['reg_day'] = df_users['reg_date'].dt.day

In [34]:
# Add Month 'churn_date'
df_users['churn_month'] = df_users['churn_date'].dt.month

In [35]:
# Add Day 'churn_date'
df_users['churn_day'] = df_users['churn_date'].dt.day

In [36]:
df_users

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date,full_name,reg_month,reg_day,churn_month,churn_day
0,1000,anamaria,bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",2018-12-24 00:00:00+00:00,ultimate,NaT,anamaria_bauer,12,24,NaN,NaN
1,1001,mickey,wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",2018-08-13 00:00:00+00:00,surf,NaT,mickey_wilkerson,8,13,NaN,NaN
2,1002,carlee,hoffman,36,"las_vegas_henderson_paradise,_nv_msa",2018-10-21 00:00:00+00:00,surf,NaT,carlee_hoffman,10,21,NaN,NaN
3,1003,reynaldo,jenkins,52,"tulsa,_ok_msa",2018-01-28 00:00:00+00:00,surf,NaT,reynaldo_jenkins,1,28,NaN,NaN
4,1004,leonila,thompson,40,"seattle_tacoma_bellevue,_wa_msa",2018-05-23 00:00:00+00:00,surf,NaT,leonila_thompson,5,23,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,fidel,sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",2018-09-04 00:00:00+00:00,surf,NaT,fidel_sharpe,9,4,NaN,NaN
496,1496,ariel,shepherd,49,"new_orleans_metairie,_la_msa",2018-02-20 00:00:00+00:00,surf,NaT,ariel_shepherd,2,20,NaN,NaN
497,1497,donte,barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",2018-12-10 00:00:00+00:00,ultimate,NaT,donte_barrera,12,10,NaN,NaN
498,1498,scot,williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",2018-02-04 00:00:00+00:00,surf,NaT,scot_williamson,2,4,NaN,NaN


In [37]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "users_feature.csv"

df_users.to_csv(processed_path, index=False)

### 4.2 Calls activity

In [38]:
df_calls

,call_id,user_id,call_date,duration,month,day
0,1000_93,1000,2018-12-27 00:00:00+00:00,9.0,12,27
1,1000_145,1000,2018-12-27 00:00:00+00:00,14.0,12,27
2,1000_247,1000,2018-12-27 00:00:00+00:00,15.0,12,27
3,1000_309,1000,2018-12-28 00:00:00+00:00,6.0,12,28
4,1000_380,1000,2018-12-30 00:00:00+00:00,5.0,12,30
...,...,...,...,...,...,...
137730,1499_199,1499,2018-11-21 00:00:00+00:00,9.0,11,21
137731,1499_200,1499,2018-10-20 00:00:00+00:00,11.0,10,20
137732,1499_201,1499,2018-09-21 00:00:00+00:00,9.0,9,21
137733,1499_202,1499,2018-10-10 00:00:00+00:00,1.0,10,10


#### 4.2.1 Calls amount and duration per user

In [39]:
df_calls_user_total = df_calls.groupby('user_id')['call_id'].count().reset_index()
df_calls_user_total = df_calls_user_total.rename(columns={'call_id': 'total_calls'})
df_calls_user_total

,user_id,total_calls
0,1000,16
1,1001,261
2,1002,113
3,1003,149
4,1004,370
...,...,...
476,1495,253
477,1496,195
478,1497,54
479,1498,451


In [40]:
df_duration_user_total = df_calls.groupby('user_id')['duration'].sum().reset_index()
df_duration_user_total = df_duration_user_total.rename(columns={'duration': 'total_duration'})
df_duration_user_total

,user_id,total_duration
0,1000,124.0
1,1001,1728.0
2,1002,829.0
3,1003,1104.0
4,1004,2772.0
...,...,...
476,1495,1765.0
477,1496,1455.0
478,1497,300.0
479,1498,3211.0


In [41]:
df_calls_duration_total = pd.merge(df_calls_user_total, df_duration_user_total, how='outer', on='user_id')
df_calls_duration_total

,user_id,total_calls,total_duration
0,1000,16,124.0
1,1001,261,1728.0
2,1002,113,829.0
3,1003,149,1104.0
4,1004,370,2772.0
...,...,...,...
476,1495,253,1765.0
477,1496,195,1455.0
478,1497,54,300.0
479,1498,451,3211.0


In [42]:
df_calls_duration_total = df_calls_duration_total.merge(df_users[['user_id', 'plan']], on='user_id', how='left')
df_calls_duration_total

,user_id,total_calls,total_duration,plan
0,1000,16,124.0,ultimate
1,1001,261,1728.0,surf
2,1002,113,829.0,surf
3,1003,149,1104.0,surf
4,1004,370,2772.0,surf
...,...,...,...,...
476,1495,253,1765.0,surf
477,1496,195,1455.0,surf
478,1497,54,300.0,ultimate
479,1498,451,3211.0,surf


In [43]:
df_calls_duration_total = df_calls_duration_total.merge(df_users[['user_id', 'city']], on='user_id', how='left')
df_calls_duration_total

,user_id,total_calls,total_duration,plan,city
0,1000,16,124.0,ultimate,"atlanta_sandy_springs_roswell,_ga_msa"
1,1001,261,1728.0,surf,"seattle_tacoma_bellevue,_wa_msa"
2,1002,113,829.0,surf,"las_vegas_henderson_paradise,_nv_msa"
3,1003,149,1104.0,surf,"tulsa,_ok_msa"
4,1004,370,2772.0,surf,"seattle_tacoma_bellevue,_wa_msa"
...,...,...,...,...,...
476,1495,253,1765.0,surf,"new_york_newark_jersey_city,_ny_nj_pa_msa"
477,1496,195,1455.0,surf,"new_orleans_metairie,_la_msa"
478,1497,54,300.0,ultimate,"los_angeles_long_beach_anaheim,_ca_msa"
479,1498,451,3211.0,surf,"new_york_newark_jersey_city,_ny_nj_pa_msa"


In [44]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "total" / "calls_duration_total.csv"

df_calls_duration_total.to_csv(processed_path, index=False)

### 4.3 Messages activity

In [45]:
df_messages

,message_id,user_id,message_date,month,day
0,1000_125,1000,2018-12-27 00:00:00+00:00,12,27
1,1000_160,1000,2018-12-31 00:00:00+00:00,12,31
2,1000_223,1000,2018-12-31 00:00:00+00:00,12,31
3,1000_251,1000,2018-12-27 00:00:00+00:00,12,27
4,1000_255,1000,2018-12-26 00:00:00+00:00,12,26
...,...,...,...,...,...
76046,1497_526,1497,2018-12-24 00:00:00+00:00,12,24
76047,1497_536,1497,2018-12-24 00:00:00+00:00,12,24
76048,1497_547,1497,2018-12-31 00:00:00+00:00,12,31
76049,1497_558,1497,2018-12-24 00:00:00+00:00,12,24


#### 4.3.1 Messages amount per user

In [46]:
# Number of messages per user
df_messages_user_total = df_messages.groupby('user_id')['message_id'].count().reset_index()
df_messages_user_total = df_messages_user_total.rename(columns={'message_id': 'total_message'})
df_messages_user_total

,user_id,total_message
0,1000,11
1,1001,207
2,1002,88
3,1003,50
4,1004,177
...,...,...
397,1491,409
398,1492,108
399,1494,174
400,1496,65


In [47]:
df_messages_user_total = df_messages_user_total.merge(df_users[['user_id', 'plan']], on='user_id', how='left')
df_messages_user_total

,user_id,total_message,plan
0,1000,11,ultimate
1,1001,207,surf
2,1002,88,surf
3,1003,50,surf
4,1004,177,surf
...,...,...,...
397,1491,409,surf
398,1492,108,surf
399,1494,174,surf
400,1496,65,surf


In [48]:
df_messages_user_total = df_messages_user_total.merge(df_users[['user_id', 'city']], on='user_id', how='left')
df_messages_user_total

,user_id,total_message,plan,city
0,1000,11,ultimate,"atlanta_sandy_springs_roswell,_ga_msa"
1,1001,207,surf,"seattle_tacoma_bellevue,_wa_msa"
2,1002,88,surf,"las_vegas_henderson_paradise,_nv_msa"
3,1003,50,surf,"tulsa,_ok_msa"
4,1004,177,surf,"seattle_tacoma_bellevue,_wa_msa"
...,...,...,...,...
397,1491,409,surf,"grand_rapids_kentwood,_mi_msa"
398,1492,108,surf,"portland_vancouver_hillsboro,_or_wa_msa"
399,1494,174,surf,"new_york_newark_jersey_city,_ny_nj_pa_msa"
400,1496,65,surf,"new_orleans_metairie,_la_msa"


In [49]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "total" / "messages_total.csv"

df_messages_user_total.to_csv(processed_path, index=False)

### 4.4 Internet activity

In [50]:
df_internet

,session_id,user_id,session_date,mb_used,month,day
0,1000_13,1000,2018-12-29 00:00:00+00:00,89.86,12,29
1,1000_204,1000,2018-12-31 00:00:00+00:00,0.00,12,31
2,1000_379,1000,2018-12-28 00:00:00+00:00,660.40,12,28
3,1000_413,1000,2018-12-26 00:00:00+00:00,270.99,12,26
4,1000_442,1000,2018-12-27 00:00:00+00:00,880.22,12,27
...,...,...,...,...,...,...
104820,1499_215,1499,2018-10-20 00:00:00+00:00,218.06,10,20
104821,1499_216,1499,2018-12-30 00:00:00+00:00,304.72,12,30
104822,1499_217,1499,2018-09-22 00:00:00+00:00,292.75,9,22
104823,1499_218,1499,2018-12-07 00:00:00+00:00,0.00,12,7


#### 4.4.1 Internet sessions and traffic per user

In [51]:
# Number of sessions per user
df_sessions_user_total = df_internet.groupby('user_id')['session_id'].count().reset_index()
df_sessions_user_total = df_sessions_user_total.rename(columns={'session_id': 'total_sessions'})
df_sessions_user_total

,user_id,total_sessions
0,1000,5
1,1001,245
2,1002,124
3,1003,52
4,1004,460
...,...,...
484,1495,290
485,1496,225
486,1497,31
487,1498,657


In [52]:
# Individual web sessions are not rounded up. Instead, the monthly total is rounded up
df_traffic_user_month = df_internet.groupby(['user_id', 'month'])['mb_used'].sum().reset_index()
df_traffic_user_month = df_traffic_user_month.rename(columns={'mb_used': 'total_traffic_month'})
df_traffic_user_month['total_traffic_month'] = np.ceil(df_traffic_user_month['total_traffic_month'] / 1000) * 1000
df_traffic_user_month

,user_id,month,total_traffic_month
0,1000,12,2000.0
1,1001,8,7000.0
2,1001,9,14000.0
3,1001,10,23000.0
4,1001,11,19000.0
...,...,...,...
2272,1498,12,24000.0
2273,1499,9,13000.0
2274,1499,10,20000.0
2275,1499,11,17000.0


In [53]:
df_traffic_user_total = df_traffic_user_month.groupby(['user_id'])['total_traffic_month'].sum().reset_index()
df_traffic_user_total = df_traffic_user_total.rename(columns={'total_traffic_month': 'total_traffic'})
df_traffic_user_total

,user_id,total_traffic
0,1000,2000.0
1,1001,83000.0
2,1002,42000.0
3,1003,28000.0
4,1004,159000.0
...,...,...
484,1495,100000.0
485,1496,67000.0
486,1497,12000.0
487,1498,232000.0


In [54]:
df_sessions_traffic_total = pd.merge(df_sessions_user_total, df_traffic_user_total, how='outer', on='user_id')
df_sessions_traffic_total

,user_id,total_sessions,total_traffic
0,1000,5,2000.0
1,1001,245,83000.0
2,1002,124,42000.0
3,1003,52,28000.0
4,1004,460,159000.0
...,...,...,...
484,1495,290,100000.0
485,1496,225,67000.0
486,1497,31,12000.0
487,1498,657,232000.0


In [55]:
df_sessions_traffic_total = df_sessions_traffic_total.merge(df_users[['user_id', 'plan']], on='user_id', how='left')
df_sessions_traffic_total

,user_id,total_sessions,total_traffic,plan
0,1000,5,2000.0,ultimate
1,1001,245,83000.0,surf
2,1002,124,42000.0,surf
3,1003,52,28000.0,surf
4,1004,460,159000.0,surf
...,...,...,...,...
484,1495,290,100000.0,surf
485,1496,225,67000.0,surf
486,1497,31,12000.0,ultimate
487,1498,657,232000.0,surf


In [56]:
df_sessions_traffic_total = df_sessions_traffic_total.merge(df_users[['user_id', 'city']], on='user_id', how='left')
df_sessions_traffic_total

,user_id,total_sessions,total_traffic,plan,city
0,1000,5,2000.0,ultimate,"atlanta_sandy_springs_roswell,_ga_msa"
1,1001,245,83000.0,surf,"seattle_tacoma_bellevue,_wa_msa"
2,1002,124,42000.0,surf,"las_vegas_henderson_paradise,_nv_msa"
3,1003,52,28000.0,surf,"tulsa,_ok_msa"
4,1004,460,159000.0,surf,"seattle_tacoma_bellevue,_wa_msa"
...,...,...,...,...,...
484,1495,290,100000.0,surf,"new_york_newark_jersey_city,_ny_nj_pa_msa"
485,1496,225,67000.0,surf,"new_orleans_metairie,_la_msa"
486,1497,31,12000.0,ultimate,"los_angeles_long_beach_anaheim,_ca_msa"
487,1498,657,232000.0,surf,"new_york_newark_jersey_city,_ny_nj_pa_msa"


In [57]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "total" / "internet_total.csv"

df_sessions_traffic_total.to_csv(processed_path, index=False)

### 4.5 Revenue

#### 4.5.0 Path to data featured file

In [58]:
data_file_path = project_root / "data" / "processed" / "feature"

df_calls_feature = load_dataset_from_csv(data_file_path, "calls_feature.csv", sep=',', header='infer')
df_messages_feature = load_dataset_from_csv(data_file_path, "messages_feature.csv", sep=',', header='infer')
df_internet_feature = load_dataset_from_csv(data_file_path, "internet_feature.csv", sep=',', header='infer')
df_users_feature = load_dataset_from_csv(data_file_path, "users_feature.csv", sep=',', header='infer')

#### 4.5.1 Total revenue per user

In [59]:
df_revenue_total = df_users_feature.loc[:, ['user_id', 'full_name', 'age', 'city', 'reg_day', 'reg_month', 'churn_day', 'churn_month', 'plan']]
df_revenue_total

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan
0,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate
1,1001,mickey_wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",13,8,NaN,NaN,surf
2,1002,carlee_hoffman,36,"las_vegas_henderson_paradise,_nv_msa",21,10,NaN,NaN,surf
3,1003,reynaldo_jenkins,52,"tulsa,_ok_msa",28,1,NaN,NaN,surf
4,1004,leonila_thompson,40,"seattle_tacoma_bellevue,_wa_msa",23,5,NaN,NaN,surf
...,...,...,...,...,...,...,...,...,...
495,1495,fidel_sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,9,NaN,NaN,surf
496,1496,ariel_shepherd,49,"new_orleans_metairie,_la_msa",20,2,NaN,NaN,surf
497,1497,donte_barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",10,12,NaN,NaN,ultimate
498,1498,scot_williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,2,NaN,NaN,surf


In [60]:
df_revenue_total = df_revenue_total.merge(df_calls_duration_total[['user_id', 'total_duration']], on='user_id', how='left')
df_revenue_total = df_revenue_total.rename(columns={'total_duration': 'total_calls_duration'})
df_revenue_total

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,total_calls_duration
0,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,124.0
1,1001,mickey_wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",13,8,NaN,NaN,surf,1728.0
2,1002,carlee_hoffman,36,"las_vegas_henderson_paradise,_nv_msa",21,10,NaN,NaN,surf,829.0
3,1003,reynaldo_jenkins,52,"tulsa,_ok_msa",28,1,NaN,NaN,surf,1104.0
4,1004,leonila_thompson,40,"seattle_tacoma_bellevue,_wa_msa",23,5,NaN,NaN,surf,2772.0
...,...,...,...,...,...,...,...,...,...,...
495,1495,fidel_sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,9,NaN,NaN,surf,1765.0
496,1496,ariel_shepherd,49,"new_orleans_metairie,_la_msa",20,2,NaN,NaN,surf,1455.0
497,1497,donte_barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",10,12,NaN,NaN,ultimate,300.0
498,1498,scot_williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,2,NaN,NaN,surf,3211.0


In [61]:
df_revenue_total = df_revenue_total.merge(df_sessions_traffic_total[['user_id', 'total_traffic']], on='user_id', how='left')
df_revenue_total = df_revenue_total.rename(columns={'total_traffic': 'total_traffic_usage'})
df_revenue_total

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,total_calls_duration,total_traffic_usage
0,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,124.0,2000.0
1,1001,mickey_wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",13,8,NaN,NaN,surf,1728.0,83000.0
2,1002,carlee_hoffman,36,"las_vegas_henderson_paradise,_nv_msa",21,10,NaN,NaN,surf,829.0,42000.0
3,1003,reynaldo_jenkins,52,"tulsa,_ok_msa",28,1,NaN,NaN,surf,1104.0,28000.0
4,1004,leonila_thompson,40,"seattle_tacoma_bellevue,_wa_msa",23,5,NaN,NaN,surf,2772.0,159000.0
...,...,...,...,...,...,...,...,...,...,...,...
495,1495,fidel_sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,9,NaN,NaN,surf,1765.0,100000.0
496,1496,ariel_shepherd,49,"new_orleans_metairie,_la_msa",20,2,NaN,NaN,surf,1455.0,67000.0
497,1497,donte_barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",10,12,NaN,NaN,ultimate,300.0,12000.0
498,1498,scot_williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,2,NaN,NaN,surf,3211.0,232000.0


In [62]:
df_revenue_total = df_revenue_total.merge(df_messages_user_total[['user_id', 'total_message']], on='user_id', how='left')
df_revenue_total = df_revenue_total.rename(columns={'total_message': 'total_messages_amount'})
df_revenue_total

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,total_calls_duration,total_traffic_usage,total_messages_amount
0,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,124.0,2000.0,11.0
1,1001,mickey_wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",13,8,NaN,NaN,surf,1728.0,83000.0,207.0
2,1002,carlee_hoffman,36,"las_vegas_henderson_paradise,_nv_msa",21,10,NaN,NaN,surf,829.0,42000.0,88.0
3,1003,reynaldo_jenkins,52,"tulsa,_ok_msa",28,1,NaN,NaN,surf,1104.0,28000.0,50.0
4,1004,leonila_thompson,40,"seattle_tacoma_bellevue,_wa_msa",23,5,NaN,NaN,surf,2772.0,159000.0,177.0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,fidel_sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,9,NaN,NaN,surf,1765.0,100000.0,NaN
496,1496,ariel_shepherd,49,"new_orleans_metairie,_la_msa",20,2,NaN,NaN,surf,1455.0,67000.0,65.0
497,1497,donte_barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",10,12,NaN,NaN,ultimate,300.0,12000.0,50.0
498,1498,scot_williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,2,NaN,NaN,surf,3211.0,232000.0,NaN


In [63]:
df_revenue_total['total_calls_revenue'] = np.round(df_revenue_total.apply(lambda row: revenue(row, revenue_type='duration', columns=['total_calls_duration', 'total_traffic_usage', 'total_messages_amount']), axis=1), 3)
df_revenue_total

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,total_calls_duration,total_traffic_usage,total_messages_amount,total_calls_revenue
0,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,124.0,2000.0,11.0,70.00
1,1001,mickey_wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",13,8,NaN,NaN,surf,1728.0,83000.0,207.0,56.84
2,1002,carlee_hoffman,36,"las_vegas_henderson_paradise,_nv_msa",21,10,NaN,NaN,surf,829.0,42000.0,88.0,29.87
3,1003,reynaldo_jenkins,52,"tulsa,_ok_msa",28,1,NaN,NaN,surf,1104.0,28000.0,50.0,38.12
4,1004,leonila_thompson,40,"seattle_tacoma_bellevue,_wa_msa",23,5,NaN,NaN,surf,2772.0,159000.0,177.0,88.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,fidel_sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,9,NaN,NaN,surf,1765.0,100000.0,NaN,57.95
496,1496,ariel_shepherd,49,"new_orleans_metairie,_la_msa",20,2,NaN,NaN,surf,1455.0,67000.0,65.0,48.65
497,1497,donte_barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",10,12,NaN,NaN,ultimate,300.0,12000.0,50.0,70.00
498,1498,scot_williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,2,NaN,NaN,surf,3211.0,232000.0,NaN,101.33


In [64]:
df_revenue_total['total_messages_revenue'] = np.round(df_revenue_total.apply(lambda row: revenue(row, revenue_type='messages', columns=['total_calls_duration', 'total_traffic_usage', 'total_messages_amount']), axis=1), 3)
df_revenue_total

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,total_calls_duration,total_traffic_usage,total_messages_amount,total_calls_revenue,total_messages_revenue
0,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,124.0,2000.0,11.0,70.00,70.00
1,1001,mickey_wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",13,8,NaN,NaN,surf,1728.0,83000.0,207.0,56.84,24.71
2,1002,carlee_hoffman,36,"las_vegas_henderson_paradise,_nv_msa",21,10,NaN,NaN,surf,829.0,42000.0,88.0,29.87,21.14
3,1003,reynaldo_jenkins,52,"tulsa,_ok_msa",28,1,NaN,NaN,surf,1104.0,28000.0,50.0,38.12,20.00
4,1004,leonila_thompson,40,"seattle_tacoma_bellevue,_wa_msa",23,5,NaN,NaN,surf,2772.0,159000.0,177.0,88.16,23.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,fidel_sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,9,NaN,NaN,surf,1765.0,100000.0,NaN,57.95,20.00
496,1496,ariel_shepherd,49,"new_orleans_metairie,_la_msa",20,2,NaN,NaN,surf,1455.0,67000.0,65.0,48.65,20.45
497,1497,donte_barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",10,12,NaN,NaN,ultimate,300.0,12000.0,50.0,70.00,70.00
498,1498,scot_williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,2,NaN,NaN,surf,3211.0,232000.0,NaN,101.33,20.00


In [65]:
df_revenue_total['total_traffic_revenue'] = np.round(df_revenue_total.apply(lambda row: revenue(row, revenue_type='traffic', columns=['total_calls_duration', 'total_traffic_usage', 'total_messages_amount']), axis=1), 3)
df_revenue_total

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,total_calls_duration,total_traffic_usage,total_messages_amount,total_calls_revenue,total_messages_revenue,total_traffic_revenue
0,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,124.0,2000.0,11.0,70.00,70.00,70.0
1,1001,mickey_wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",13,8,NaN,NaN,surf,1728.0,83000.0,207.0,56.84,24.71,696.4
2,1002,carlee_hoffman,36,"las_vegas_henderson_paradise,_nv_msa",21,10,NaN,NaN,surf,829.0,42000.0,88.0,29.87,21.14,286.4
3,1003,reynaldo_jenkins,52,"tulsa,_ok_msa",28,1,NaN,NaN,surf,1104.0,28000.0,50.0,38.12,20.00,146.4
4,1004,leonila_thompson,40,"seattle_tacoma_bellevue,_wa_msa",23,5,NaN,NaN,surf,2772.0,159000.0,177.0,88.16,23.81,1456.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,fidel_sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,9,NaN,NaN,surf,1765.0,100000.0,NaN,57.95,20.00,866.4
496,1496,ariel_shepherd,49,"new_orleans_metairie,_la_msa",20,2,NaN,NaN,surf,1455.0,67000.0,65.0,48.65,20.45,536.4
497,1497,donte_barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",10,12,NaN,NaN,ultimate,300.0,12000.0,50.0,70.00,70.00,70.0
498,1498,scot_williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,2,NaN,NaN,surf,3211.0,232000.0,NaN,101.33,20.00,2186.4


In [66]:
df_revenue_total['total_services_revenue'] = np.round(df_revenue_total.apply(lambda row: revenue(row, revenue_type='total', columns=['total_calls_duration', 'total_traffic_usage', 'total_messages_amount']), axis=1), 3)
df_revenue_total

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,total_calls_duration,total_traffic_usage,total_messages_amount,total_calls_revenue,total_messages_revenue,total_traffic_revenue,total_services_revenue
0,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,124.0,2000.0,11.0,70.00,70.00,70.0,70.00
1,1001,mickey_wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",13,8,NaN,NaN,surf,1728.0,83000.0,207.0,56.84,24.71,696.4,737.95
2,1002,carlee_hoffman,36,"las_vegas_henderson_paradise,_nv_msa",21,10,NaN,NaN,surf,829.0,42000.0,88.0,29.87,21.14,286.4,297.41
3,1003,reynaldo_jenkins,52,"tulsa,_ok_msa",28,1,NaN,NaN,surf,1104.0,28000.0,50.0,38.12,20.00,146.4,164.52
4,1004,leonila_thompson,40,"seattle_tacoma_bellevue,_wa_msa",23,5,NaN,NaN,surf,2772.0,159000.0,177.0,88.16,23.81,1456.4,1528.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,fidel_sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,9,NaN,NaN,surf,1765.0,100000.0,NaN,57.95,20.00,866.4,904.35
496,1496,ariel_shepherd,49,"new_orleans_metairie,_la_msa",20,2,NaN,NaN,surf,1455.0,67000.0,65.0,48.65,20.45,536.4,565.50
497,1497,donte_barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",10,12,NaN,NaN,ultimate,300.0,12000.0,50.0,70.00,70.00,70.0,70.00
498,1498,scot_williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,2,NaN,NaN,surf,3211.0,232000.0,NaN,101.33,20.00,2186.4,2267.73


In [67]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "total" / "revenue_total.csv"

df_revenue_total.to_csv(processed_path, index=False)

#### 4.5.2 Periodic activity per user

In [68]:
df_calls_duration_periodic = df_calls_feature.groupby(['user_id', 'month', 'day'])['duration'].sum().reset_index()
df_calls_duration_periodic

,user_id,month,day,duration
0,1000,12,26,15.0
1,1000,12,27,50.0
2,1000,12,28,31.0
3,1000,12,29,2.0
4,1000,12,30,5.0
...,...,...,...,...
51954,1499,12,27,9.0
51955,1499,12,28,9.0
51956,1499,12,29,22.0
51957,1499,12,30,5.0


In [69]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "periodic" / "calls_duration_periodic.csv"

df_calls_duration_periodic.to_csv(processed_path, index=False)

In [70]:
df_messages_periodic = df_messages_feature.groupby(['user_id', 'month', 'day'])['message_id'].count().reset_index()
df_messages_periodic = df_messages_periodic.rename(columns={'message_id': 'message_count'})
df_messages_periodic

,user_id,month,day,message_count
0,1000,12,25,1
1,1000,12,26,1
2,1000,12,27,3
3,1000,12,28,1
4,1000,12,29,1
...,...,...,...,...
33238,1497,12,27,4
33239,1497,12,28,2
33240,1497,12,29,5
33241,1497,12,30,1


In [71]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "periodic" / "messages_periodic.csv"

df_messages_periodic.to_csv(processed_path, index=False)

In [72]:
df_internet_periodic = df_internet_feature.groupby(['user_id', 'month', 'day'])['mb_used'].sum().reset_index()
df_internet_periodic

,user_id,month,day,mb_used
0,1000,12,26,270.99
1,1000,12,27,880.22
2,1000,12,28,660.40
3,1000,12,29,89.86
4,1000,12,31,0.00
...,...,...,...,...
49028,1499,12,25,924.14
49029,1499,12,26,823.03
49030,1499,12,29,2385.78
49031,1499,12,30,304.72


In [73]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "periodic" / "internet_periodic.csv"

df_internet_periodic.to_csv(processed_path, index=False)

#### 4.5.3 Periodic revenue per user

In [74]:
df_revenue_periodic = df_users.loc[:, ['user_id', 'full_name', 'age', 'city', 'reg_day', 'reg_month', 'churn_day', 'churn_month', 'plan']]
df_revenue_periodic

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan
0,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate
1,1001,mickey_wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",13,8,NaN,NaN,surf
2,1002,carlee_hoffman,36,"las_vegas_henderson_paradise,_nv_msa",21,10,NaN,NaN,surf
3,1003,reynaldo_jenkins,52,"tulsa,_ok_msa",28,1,NaN,NaN,surf
4,1004,leonila_thompson,40,"seattle_tacoma_bellevue,_wa_msa",23,5,NaN,NaN,surf
...,...,...,...,...,...,...,...,...,...
495,1495,fidel_sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,9,NaN,NaN,surf
496,1496,ariel_shepherd,49,"new_orleans_metairie,_la_msa",20,2,NaN,NaN,surf
497,1497,donte_barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",10,12,NaN,NaN,ultimate
498,1498,scot_williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",4,2,NaN,NaN,surf


In [75]:
df_revenue_periodic = df_revenue_periodic.merge(df_calls_duration_periodic[['user_id', 'month', 'day', 'duration']], on='user_id', how='outer')
df_revenue_periodic = df_revenue_periodic.rename(columns={'duration': 'calls_duration'})
df_revenue_periodic

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,month,day,calls_duration
0,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,12.0,26.0,15.0
1,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,12.0,27.0,50.0
2,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,12.0,28.0,31.0
3,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,12.0,29.0,2.0
4,1000,anamaria_bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",24,12,NaN,NaN,ultimate,12.0,30.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
51973,1499,shena_dickson,37,"orlando_kissimmee_sanford,_fl_msa",6,5,NaN,NaN,surf,12.0,27.0,9.0
51974,1499,shena_dickson,37,"orlando_kissimmee_sanford,_fl_msa",6,5,NaN,NaN,surf,12.0,28.0,9.0
51975,1499,shena_dickson,37,"orlando_kissimmee_sanford,_fl_msa",6,5,NaN,NaN,surf,12.0,29.0,22.0
51976,1499,shena_dickson,37,"orlando_kissimmee_sanford,_fl_msa",6,5,NaN,NaN,surf,12.0,30.0,5.0


In [76]:
df_revenue_periodic = df_revenue_periodic.merge(df_messages_periodic[['user_id', 'month', 'day', 'message_count']], on=['user_id', 'month', 'day'], how='outer')
df_revenue_periodic = df_revenue_periodic.rename(columns={'message_count': 'messages_amount'})
df_revenue_periodic

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,month,day,calls_duration,messages_amount
0,1000,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,12.0,25.0,NaN,1.0
1,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,26.0,15.0,1.0
2,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,27.0,50.0,3.0
3,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,28.0,31.0,1.0
4,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,29.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57623,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,27.0,9.0,NaN
57624,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,28.0,9.0,NaN
57625,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,29.0,22.0,NaN
57626,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,30.0,5.0,NaN


In [77]:
dict_user_to_name = df_revenue_periodic.dropna(subset=['full_name']).drop_duplicates('user_id').set_index('user_id')['full_name'].to_dict()
df_revenue_periodic['full_name'] = df_revenue_periodic['full_name'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_name))


In [78]:
dict_user_to_age = df_revenue_periodic.dropna(subset=['age']).drop_duplicates('user_id').set_index('user_id')['age'].to_dict()
df_revenue_periodic['age'] = df_revenue_periodic['age'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_age))


In [79]:
dict_user_to_city = df_revenue_periodic.dropna(subset=['city']).drop_duplicates('user_id').set_index('user_id')['city'].to_dict()
df_revenue_periodic['city'] = df_revenue_periodic['city'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_city))


In [80]:
dict_user_to_regday = df_revenue_periodic.dropna(subset=['reg_day']).drop_duplicates('user_id').set_index('user_id')['reg_day'].to_dict()
df_revenue_periodic['reg_day'] = df_revenue_periodic['reg_day'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_regday))


In [81]:
dict_user_to_regmonth = df_revenue_periodic.dropna(subset=['reg_month']).drop_duplicates('user_id').set_index('user_id')['reg_month'].to_dict()
df_revenue_periodic['reg_month'] = df_revenue_periodic['reg_month'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_regmonth))


In [82]:
dict_user_to_churnday = df_revenue_periodic.dropna(subset=['churn_day']).drop_duplicates('user_id').set_index('user_id')['churn_day'].to_dict()
df_revenue_periodic['churn_day'] = df_revenue_periodic['churn_day'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_churnday))


In [83]:
dict_user_to_churnmonth = df_revenue_periodic.dropna(subset=['churn_month']).drop_duplicates('user_id').set_index('user_id')['churn_month'].to_dict()
df_revenue_periodic['churn_month'] = df_revenue_periodic['churn_month'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_churnmonth))


In [84]:
dict_user_to_plan = df_revenue_periodic.dropna(subset=['plan']).drop_duplicates('user_id').set_index('user_id')['plan'].to_dict()
df_revenue_periodic['plan'] = df_revenue_periodic['plan'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_plan))
df_revenue_periodic

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,month,day,calls_duration,messages_amount
0,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,25.0,NaN,1.0
1,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,26.0,15.0,1.0
2,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,27.0,50.0,3.0
3,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,28.0,31.0,1.0
4,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,29.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57623,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,27.0,9.0,NaN
57624,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,28.0,9.0,NaN
57625,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,29.0,22.0,NaN
57626,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,30.0,5.0,NaN


In [85]:
df_revenue_periodic = df_revenue_periodic.merge(df_internet_periodic[['user_id', 'month', 'day', 'mb_used']], on=['user_id', 'month', 'day'], how='outer')
df_revenue_periodic = df_revenue_periodic.rename(columns={'mb_used': 'traffic_usage'})
df_revenue_periodic

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,month,day,calls_duration,messages_amount,traffic_usage
0,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,25.0,NaN,1.0,NaN
1,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,26.0,15.0,1.0,270.99
2,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,27.0,50.0,3.0,880.22
3,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,28.0,31.0,1.0,660.40
4,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,29.0,2.0,1.0,89.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61223,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,27.0,9.0,NaN,NaN
61224,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,28.0,9.0,NaN,NaN
61225,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,29.0,22.0,NaN,2385.78
61226,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,30.0,5.0,NaN,304.72


In [86]:
dict_user_to_name = df_revenue_periodic.dropna(subset=['full_name']).drop_duplicates('user_id').set_index('user_id')['full_name'].to_dict()
df_revenue_periodic['full_name'] = df_revenue_periodic['full_name'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_name))


In [87]:
dict_user_to_age = df_revenue_periodic.dropna(subset=['age']).drop_duplicates('user_id').set_index('user_id')['age'].to_dict()
df_revenue_periodic['age'] = df_revenue_periodic['age'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_age))


In [88]:
dict_user_to_city = df_revenue_periodic.dropna(subset=['city']).drop_duplicates('user_id').set_index('user_id')['city'].to_dict()
df_revenue_periodic['city'] = df_revenue_periodic['city'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_city))


In [89]:
dict_user_to_regday = df_revenue_periodic.dropna(subset=['reg_day']).drop_duplicates('user_id').set_index('user_id')['reg_day'].to_dict()
df_revenue_periodic['reg_day'] = df_revenue_periodic['reg_day'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_regday))


In [90]:
dict_user_to_regmonth = df_revenue_periodic.dropna(subset=['reg_month']).drop_duplicates('user_id').set_index('user_id')['reg_month'].to_dict()
df_revenue_periodic['reg_month'] = df_revenue_periodic['reg_month'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_regmonth))


In [91]:
dict_user_to_churnday = df_revenue_periodic.dropna(subset=['churn_day']).drop_duplicates('user_id').set_index('user_id')['churn_day'].to_dict()
df_revenue_periodic['churn_day'] = df_revenue_periodic['churn_day'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_churnday))


In [92]:
dict_user_to_churnmonth = df_revenue_periodic.dropna(subset=['churn_month']).drop_duplicates('user_id').set_index('user_id')['churn_month'].to_dict()
df_revenue_periodic['churn_month'] = df_revenue_periodic['churn_month'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_churnmonth))


In [93]:
dict_user_to_plan = df_revenue_periodic.dropna(subset=['plan']).drop_duplicates('user_id').set_index('user_id')['plan'].to_dict()
df_revenue_periodic['plan'] = df_revenue_periodic['plan'].fillna(df_revenue_periodic['user_id'].map(dict_user_to_plan))
df_revenue_periodic

,user_id,full_name,age,city,reg_day,reg_month,churn_day,churn_month,plan,month,day,calls_duration,messages_amount,traffic_usage
0,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,25.0,NaN,1.0,NaN
1,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,26.0,15.0,1.0,270.99
2,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,27.0,50.0,3.0,880.22
3,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,28.0,31.0,1.0,660.40
4,1000,anamaria_bauer,45.0,"atlanta_sandy_springs_roswell,_ga_msa",24.0,12.0,NaN,NaN,ultimate,12.0,29.0,2.0,1.0,89.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61223,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,27.0,9.0,NaN,NaN
61224,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,28.0,9.0,NaN,NaN
61225,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,29.0,22.0,NaN,2385.78
61226,1499,shena_dickson,37.0,"orlando_kissimmee_sanford,_fl_msa",6.0,5.0,NaN,NaN,surf,12.0,30.0,5.0,NaN,304.72


In [115]:
cols = ['calls_duration', 'traffic_usage', 'messages_amount']

calls_revenue_fn = partial(revenue, revenue_type='duration', columns=cols)
messages_revenue_fn = partial(revenue, revenue_type='messages', columns=cols)
traffic_revenue_fn = partial(revenue, revenue_type='traffic', columns=cols)
total_revenue_fn = partial(revenue, revenue_type='total', columns=cols)


In [116]:
df_revenue_periodic['calls_revenue'] = df_revenue_periodic.apply(calls_revenue_fn, axis=1)
df_revenue_periodic['messages_revenue'] = df_revenue_periodic.apply(messages_revenue_fn, axis=1)
df_revenue_periodic['traffic_revenue'] = df_revenue_periodic.apply(traffic_revenue_fn, axis=1)
df_revenue_periodic['total_revenue'] = df_revenue_periodic.apply(total_revenue_fn, axis=1)


In [117]:
df_revenue_monthly = (
    df_revenue_periodic.groupby('month', observed=False)[
        ['calls_revenue', 'messages_revenue', 'traffic_revenue', 'total_revenue']
    ]
    .sum()
    .reset_index()
)

df_revenue_monthly

,month,calls_revenue,messages_revenue,traffic_revenue,total_revenue
0,1.0,3810.0,3810.0,3810.0,3810.0
1,2.0,14170.0,14170.0,14170.0,14170.0
2,3.0,28550.0,28550.0,28550.0,28550.0
3,4.0,53560.0,53560.0,53560.0,53560.0
4,5.0,91870.0,91870.0,91870.0,91870.0
5,6.0,129170.0,129170.0,129170.0,129170.0
6,7.0,174510.0,174510.0,174510.0,174510.0
7,8.0,224780.0,224780.0,224780.0,224780.0
8,9.0,266190.0,266190.0,266190.0,266190.0
9,10.0,338560.0,338560.0,338560.0,338560.0


In [118]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "periodic" / "revenue_monthly.csv"

df_revenue_monthly.to_csv(processed_path, index=False)